In [ ]:
from pandas import DataFrame
import json
from lyricsgenius import Genius
from utils import ArtistSongs, Artists, Charts, Songs, Lyrics
from utils.Charts import Chart, ChartList
import os

# Notebook para la extracción de los datos y creación del dataset principal

Este notebook  se centra en la extracción de datos de canciones utilizando la API de Genius. 

El objetivo principal es recopilar información relevante sobre canciones, incluyendo sus letras, artista, entre otros, y almacenarlas en un formato estructurado para su posterior análisis y uso con los modelos propuestos.

In [ ]:
#Configuración de la API: Se inicializa la conexión con la API de Genius utilizando el token proporcionado.
Artists.genius = genius
ArtistSongs.genius = genius
Charts.genius = genius
Songs.genius = genius
Lyrics.genius = genius

In [ ]:
#Recolección de Datos: Se descargan los datos de las listas de reproducción, canciones, letras y detalles de los artistas.
Charts.chart_download()
ArtistSongs.load_from_charts()
Lyrics.load_from_artist_songs()

In [ ]:
#Mapeo de Información: Se crea una función para mapear la información de cada canción a un diccionario estándar.
def id(song) -> int:
    return song._id

def name(song) -> str:
    return song.title

def artist(song) -> str:
    return song.artist

def artist_id(song) -> int:
    song_json = song.to_json()
    song_json = json.loads(song_json)
    return song_json.get('primary_artist', {}).get('id', None)

def language(song) -> str:
    song_json = song.to_json()
    song_json = json.loads(song_json)
    return song_json.get('language', None)

def lyrics(song) -> str:
    lyrics = Lyrics.lyrics(song._id)
    print(lyrics)
    lyrics = lyrics.split('\n')[1:]
    return "".join(lyrics)

def map_song(song) -> dict:
    return {
        'ID': id(song),
        'Name': name(song),
        'Artist': artist(song),
        'Artist ID': artist_id(song),
        'Language': language(song),
        'Lyrics': lyrics(song)
    }

songs = Songs.songs()
songs_dict = list(map(map_song, songs))

In [ ]:
#Exportación de Datos: Finalmente, se guarda toda la información recopilada en un archivo CSV para su posterior análisis.
directory = 'datasets'
if not os.path.exists(directory):
    os.makedirs(directory)
    
df = DataFrame(songs_dict)
df.to_csv('datasets/raw_songs.csv')